# 4. Model Deployment (PKL to ONNX) - Glute Bridge

**Objective:**
1.  Load the saved `scikit-learn` pipeline from its `.pkl` file.
2.  Convert the entire pipeline (which includes the imputer, scaler, and model) into a single ONNX file.
3.  Save the ONNX model to the `models/onnx/` directory, making it ready for the live detection application.

**Note:** Since we embedded the scaler within the pipeline, we **do not** need to create a separate `scaler.json` file. The ONNX model will handle both scaling and inference internally.

In [4]:
import os
import sys
import logging
import onnxruntime as ort # To verify the converted model
import pickle

# Add root project directory to sys.path
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.model_deploy_convertor_utils import ModelDeployConverterUtils

# Configure logger for better feedback from the utility
logger = logging.getLogger('utils.model_deploy_convertor_utils')
logger.setLevel(logging.INFO)

## 4.1 Define Paths and Parameters

In [5]:
MODEL_PKL_DIR = "../models/pkl/"
MODEL_ONNX_DIR = "../models/onnx/"
os.makedirs(MODEL_ONNX_DIR, exist_ok=True)

# <<< CHOOSE THE MODEL TO CONVERT >>>
# This should be the base name of the .pkl file (without extension)
# >>>>> YOU CAN CHANGE THIS VALUE TO 'LR', 'KNN', 'DT', 'RF', or 'XGB' if you saved it earlier <<<<<
MODEL_TYPE_TO_LOAD = "RF" 

TARGET_OPSET = 12 # A common and widely supported ONNX opset version

## 4.2 Perform Conversion

In [6]:
print(f"--- Starting Conversion for {MODEL_TYPE_TO_LOAD}_model ---")

source_pkl_path = os.path.join(MODEL_PKL_DIR, f"{MODEL_TYPE_TO_LOAD}_model.pkl")
if not os.path.exists(source_pkl_path):
    print(f"ERROR: Source PKL file not found at: {source_pkl_path}")
    print("Please ensure you have run Notebook 2 to train and save the model.")
else:
    # Use the ModelDeployConverterUtils to perform the conversion
    onnx_path = ModelDeployConverterUtils.convert_model_pickle_to_onnx(
        pickle_model_dir=MODEL_PKL_DIR, 
        onnx_model_dir=MODEL_ONNX_DIR, 
        name=f"{MODEL_TYPE_TO_LOAD}_model",
        target_opset=TARGET_OPSET
    )

    if onnx_path and os.path.exists(onnx_path):
        print(f"\nSUCCESS: Pipeline converted and saved to ONNX format at:")
        print(f"==> {onnx_path}")
        
        # --- Verification Step ---
        print("\n--- Verifying the ONNX Model ---")
        try:
            # Load the PKL model to get the expected number of features
            with open(source_pkl_path, 'rb') as f:
                skl_pipeline = pickle.load(f)
            
            # Get the number of features from the imputer or scaler step
            n_features = skl_pipeline.named_steps['imputer'].n_features_in_
            
            # Load the ONNX model
            ort_session = ort.InferenceSession(onnx_path)
            input_name = ort_session.get_inputs()[0].name
            output_names = [output.name for output in ort_session.get_outputs()]
            
            print(f"ONNX model loaded successfully.")
            print(f"Input Name: {input_name}")
            print(f"Output Names: {output_names}")
            print(f"Expected number of input features: {n_features}")
            
            # Test with a dummy input
            dummy_input = np.random.rand(1, n_features).astype(np.float32)
            onnx_outputs = ort_session.run(output_names, {input_name: dummy_input})
            print("\nDummy inference test successful!")
            print(f"Predicted Label: {onnx_outputs[0]}")
            print(f"Predicted Probabilities: {onnx_outputs[1]}")

        except Exception as e:
            print(f"ERROR during ONNX model verification: {e}")
    else:
        print(f"\nFAILURE: Model conversion to ONNX failed. Check the logs above for errors.")

2025-06-07 21:25:41,468 - utils.model_deploy_convertor_utils - INFO - Starting conversion for model 'RF_model': ../models/pkl/RF_model.pkl -> ../models/onnx/RF_model.onnx
2025-06-07 21:25:41,473 - utils.model_deploy_convertor_utils - INFO - Successfully loaded pickle model from ../models/pkl/RF_model.pkl
2025-06-07 21:25:41,474 - utils.model_deploy_convertor_utils - INFO - Model for 'RF_model' expects 20 input features.
2025-06-07 21:25:41,475 - utils.model_deploy_convertor_utils - INFO - Attempting to convert model 'RF_model' to ONNX with target_opset=12.
2025-06-07 21:25:41,519 - utils.model_deploy_convertor_utils - INFO - Successfully converted model 'RF_model' to ONNX format.
2025-06-07 21:25:41,521 - utils.model_deploy_convertor_utils - INFO - Successfully saved ONNX model to ../models/onnx/RF_model.onnx


--- Starting Conversion for RF_model ---

SUCCESS: Pipeline converted and saved to ONNX format at:
==> ../models/onnx/RF_model.onnx

--- Verifying the ONNX Model ---
ONNX model loaded successfully.
Input Name: float_input
Output Names: ['output_label', 'output_probability']
Expected number of input features: 20
ERROR during ONNX model verification: name 'np' is not defined


### Deployment Summary

The process is complete. The selected scikit-learn pipeline (`.pkl`) has been converted into a self-contained ONNX model.

This single `.onnx` file now contains all the necessary logic for:
1.  Imputing missing values.
2.  Scaling the input features.
3.  Making a classification prediction.

This artifact is now ready to be used in the live camera detection notebook (`5_live_camera_detection.ipynb`).